<a href="https://colab.research.google.com/github/ZHANG-ZHENPENG/GVI_Extraction/blob/main/GVIworkflow02_Calculate_Tree_pixel_proportion%2BWrite_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1 Calculate tree pixel + Write CSV

In [ ]:
# Fix bugs in multi-tract: Find and delete images without tree pixels
wardpath = '/content/drive/MyDrive/ExpandedTracts/Ward_X/'
all_items = os.listdir(wardpath)
subtract = [item for item in all_items if os.path.isdir(os.path.join(wardpath, item))]
print(subtract)
suffix = '*.png'
## rgb 107 , 142, 35 # the tree color in RGB
## b,g,r = 35, 142, 107 # cv2 uses BGR
class_color = [35, 142, 107]

for tract in subtract:
  savepath = wardpath+tract
  print("Find bugs in:",savepath)
  all_files = os.listdir(savepath)
  subdirectories = [item for item in all_files if os.path.isdir(os.path.join(savepath, item))]
  num_subdirectories = len(subdirectories)
  print("Number of subdirectories:", num_subdirectories)

  for i in range(0, num_subdirectories):
    grouppath = savepath+'/group0{}/vis/'.format(i+1)
    savename = tract+'group0{}GVI.csv'.format(i+1)
    print("Find bugs in:",grouppath)
    pathname=grouppath+suffix

    ImageName = []
    TreePixels = []
    for file in glob.glob(pathname):
      eachimage = cv2.imread(file)
      filteredImage = eachimage.copy()
      low = np.array(class_color);
      # Create a mask for images
      mask = cv2.inRange(filteredImage, low, low)
      filteredImage[mask == 0] = [0, 0, 0]
      filteredImage[mask != 0] = [255,255,255]
      # Calculate tree pixels for each image and save them into list.
      bwImage = cv2.cvtColor(filteredImage, cv2.COLOR_BGR2GRAY)
      if type(cv2.findNonZero(bwImage)) == type(None):
        os.remove(file)
        print("[File without Tree Pixels] deleted:",file)
    else:
      print("No bugs founded")

In [ ]:
# GVI calculate and write csv
class_color = [35, 142, 107]
wardpath = '/content/drive/MyDrive/ExpandedTracts/Ward_X/'

all_items = os.listdir(wardpath)
subtract = [item for item in all_items if os.path.isdir(os.path.join(wardpath, item))]
print(subtract)
suffix = '*.png'

for tract in subtract:
  savepath = wardpath+tract
  # Get a list of all items in the directory
  all_items = os.listdir(savepath)
  # Filter out only directories
  subdirectories = [item for item in all_items if os.path.isdir(os.path.join(savepath, item))]
  # Count the number of subdirectories
  num_subdirectories = len(subdirectories)
  print("Number of subdirectories:", num_subdirectories)

  for i in range(0, num_subdirectories):
    grouppath = savepath+'/group0{}/vis/'.format(i+1)
    savename = tract+'group0{}GVI.csv'.format(i+1)
    print("Input:",grouppath)
    print("Output:",savename)
    print("Savepath:",savepath)
    pathname=grouppath+suffix

    ImageName = []
    images = []
    for file in glob.glob(pathname):
      eachimage = cv2.imread(file)
      images.append(eachimage)

    # Get a list of TreePixels
    TreePixels = []
    for eachimage in images:
      #plt.imshow(eachimage)
      #plt.axis("off")
      #plt.show()
      filteredImage = eachimage.copy()
      low = np.array(class_color);

      # Create a mask for images
      mask = cv2.inRange(filteredImage, low, low)
      filteredImage[mask == 0] = [0, 0, 0]
      filteredImage[mask != 0] = [255,255,255]
      #plt.imshow(filteredImage)
      #plt.axis("off")
      #plt.show()

      # Calculate tree pixels for each image and save them into list.
      bwImage = cv2.cvtColor(filteredImage, cv2.COLOR_BGR2GRAY)
      #plt.imshow(bwImage)
      #plt.axis("off")
      #plt.show()
      #cv2_imshow("BW", bwImage)
      numberPixels = len(cv2.findNonZero(bwImage))
      # print(numberPixels)
      TreePixels.append(numberPixels)
      cv2.waitKey(0)

    print("TreePixelList:",TreePixels)

    # Get a list of TotalPixels
    TotalPixels = []
    for file in glob.glob(pathname):
      img = Image.open(file)
      width, height = img.size
      Total_pixels = width * height
      TotalPixels.append(Total_pixels)

    print("TotalPixelsList:",TotalPixels)

    # Get Calculate GreenViewIndex and get a list of GreenViewIndex
    GreenViewIndex=[]
    print(len(TreePixels))

    for i in range(0, len(TreePixels)):
      GVI = (TreePixels[i]/TotalPixels[i])*100
      GreenViewIndex.append(GVI)
    print("GVIList:",GreenViewIndex)

    # Get all the name of images
    from os import listdir
    from os.path import isfile, join

    FileName = [f for f in listdir(grouppath) if isfile(join(grouppath, f))]
    print("NameList:",FileName)
    Location = []
    for i in range(0, len(FileName)):
      Location.append(tract)
    print("LocationList:",Location)
    # Save GVI csv file to a specific directory
    os.chdir(savepath)

    import csv
    # write a csv file with 2 colmuns
    data_list = []

    for filename,GVI,location in zip(FileName,GreenViewIndex,Location):
      x={}
      x["ImageName"]=filename
      x["GreenViewIndex"]=GVI
      x["Location"]=location
      data_list.append(x)
    # Print to check
    print(data_list)

    # Write csv
    with open(savename,'w',newline='',encoding='UTF-8') as f_c_csv:
        writer = csv.writer(f_c_csv)
        writer.writerow(['ImageName', 'GreenViewIndex','Location'])
        for nl in data_list:
            writer.writerow(nl.values())
    print("Done")

In [ ]:
# Combining Multiple CSV Files in tract groups
savepath = '/content/drive/MyDrive/ExpandedTracts/Ward6/'

all_items = os.listdir(savepath)
subtract = [item for item in all_items if os.path.isdir(os.path.join(savepath, item))]

for tract in subtract:
  walkpath = savepath +tract
  combined_csv = savepath + tract+'GVI.csv'
  print(combined_csv)
  print(walkpath)

  # list all csv files only
  os.chdir(walkpath)
  csv_files = glob.glob('*.{}'.format('csv'))
  print(csv_files)

  import pandas as pd
  list = []

  for f in csv_files:
      list.append(pd.read_csv(f))
  # Transform panda dataframe to csv, and store it in google drive
  df_res = pd.concat(list, ignore_index=True)
  df_res.to_csv(combined_csv,index=False)

  print("Combined csv was saved:",combined_csv)

In [ ]:
# Combining Multiple tracts CSV Files together
savepath = '/content/drive/MyDrive/ExpandedTracts/'
Ward = 'Ward6'
walkpath = savepath +Ward
combined_csv = savepath + Ward+'GVI.csv'
print(combined_csv)
print(walkpath)

# list all csv files only
os.chdir(walkpath)
csv_files = glob.glob('*.{}'.format('csv'))
print(csv_files)

import pandas as pd
list = []

for f in csv_files:
    list.append(pd.read_csv(f))
df_res = pd.concat(list, ignore_index=True)
df_res.to_csv(combined_csv,index=False)

print("Combined csv was saved:",combined_csv)

2 Random verification

In [ ]:
os.chdir('/content/drive/MyDrive/Untitled folder/GroundTruthTest/')
image_inferred = 'Inferred.png'
image_truth = 'GroundTruthTest.png'

image01 = cv2.imread(image_inferred,1)
image02 = cv2.imread(image_truth,1)

# Check if each position value is equal
comparison = image01 == image02

# Count the number of True and False values
num_true = np.count_nonzero(comparison)
num_false = comparison.size - num_true

print("Number of True positions:", num_true)
print("Number of False positions:", num_false)

print(num_true/(266001*3))
884736